# Notebook Instructions

1. All the <u>code and data files</u> used in this course are available in the downloadable unit of the <u>last section of this course</u>.
2. You can run the notebook document sequentially (one cell at a time) by pressing **shift + enter**. 
3. While a cell is running, a [*] is shown on the left. After the cell is run, the output will appear on the next line.

This course is based on specific versions of python packages. You can find the details of the packages in <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank" >this manual</a>.

## Class Weights in Decision Trees

When we are building a decision tree model, it can happen that the dataset provided to the model may have very few data points for it's most important classes. In such an instance, the decision tree algorithm will try to maximize the accuracy of the most common labels. 

In order to adjust for this issue, we re-assign weights to the data points of the most important labels. This can be done in the scikit-library using the class_weight argument to the decision tree classifier. Let us take an example to illustrate this.

#### Example:

We will input raw data of ACC Ltd. stock from a csv file. The data consists of Open-High-Low-Close prices and Volume data. Predictor and target variables are created using this raw data. 

In [ ]:
import pandas as pd
df = pd.read_csv('../data_modules/ACC.csv')
df.tail()

#### Computing Technical Indicators and Daily Future Returns

We compute the values for the Average Directional Index (ADI), Relative Strength Index (RSI), and Simple Moving Average (SMA) using the TA-Lib package. These will be used as predictor variables in the decision tree model. Next, we compute the daily future returns on the close price. The code is shown below.


In [ ]:
import numpy as np
import talib as ta

# Import and filter warnings
import warnings
warnings.filterwarnings("ignore")

df['ADX'] = ta.ADX(df['HIGH'].values, df['LOW'].values,
                   df['CLOSE'].values, timeperiod=14)
df['RSI'] = ta.RSI(df['CLOSE'].values, timeperiod=14)
df['SMA'] = ta.SMA(df['CLOSE'].values, timeperiod=20)

df['Return'] = df['CLOSE'].pct_change(1).shift(-1)
df = df.dropna()

df.tail(15)

#### Categorize Returns into Multiple Classes

We define a function called 'returns_to_class' using nested If..else statement to categorize returns into multiple classes. We also specify the range for the returns for each class in this function. This function is then applied on our dataframe, df to get the multi-class target variable.


In [ ]:
def returns_to_class(df):
    if df.Return <= 0.0:
        return 0
    elif df.Return > 0.0 and df.Return < 0.02:
        return 1
    elif df.Return > 0.02 and df.Return < 0.03:
        return 2
    else:
        return 3


df['Class'] = df.apply(returns_to_class, axis=1)
df.tail(15)

#### View the Multi-Class Distribution

Once we have defined the different classes for the target variable, we can see their distribution of Returns using the groupby method. As can be observed, out of the total data points majority of them (i.e. 126 data points) belong to '0' class which signifies negative returns. On the other hand, there are only 11 and 1 datapoint belonging to the '2' and the '3' class respectively.

In [ ]:
df.groupby('Class').count()

#### Create Predictor Variables and Target Variable

Let us now define our predictors variables, X and the target variable, y for building a decision tree model.

In [ ]:
X = df[['ADX', 'RSI', 'SMA']]
y = df.Class

  
We will consider two scenarios:   

1) Building a decision tree model without applying the class weights and    
2) Building a decision tree model with class weights.


### Scenario 1 - Build a decision tree model without applying the Class weights 

In [22]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

# Split into Train and Test datasets
split_percentage = 0.8
split = int(split_percentage*len(X))
# Train data set
X_train = X[:split]
y_train = y[:split]
# Test data set
X_test = X[split:]
y_test = y[split:]

#print (X_train.shape, y_train.shape)
#print (X_test.shape, y_test.shape)

# Fit a model on train data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5)
clf = clf.fit(X_train, y_train)

# Use the trained model to make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model performance
report = classification_report(y_test, y_pred)
print(report)

# Here, the warning is occurring because some labels in y_test don't appear in y_pred. In this case, the number of observations
# for label 2 and label 3 are very few and they might not be occurring in the y_pred.

              precision    recall  f1-score   support

           0       0.76      0.97      0.85        29
           1       0.50      0.12      0.20         8
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1

    accuracy                           0.74        39
   macro avg       0.31      0.27      0.26        39
weighted avg       0.67      0.74      0.67        39



As can be seen from the output of the classification report, the decision tree algorithm tries to maximize the accuracy of the most common labels and does not give good predictions on the underrepresented labels.

### Scenario 2 - Build a decision tree model with Class Weights 

Let us use the class_weight parameter when defining the decision tree classifier to correct for the underrepresented labels.

We can assigned class_weight = 'balanced'. This re-weighting of the data points causes the classes to appear with equal frequency.

As can be seen from the output of the classification report, using class weight makes the decision tree algorithm achieve higher accuracy on the underrepresented labels which were labels '2'and '3' in this case. Although after class weights are changed, a model which otherwise shows good performance can suddenly appear to be less effective.

In [14]:
# Split into Train and Test datasets
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

split_percentage = 0.8
split = int(split_percentage*len(X))
# Train data set
X_train = X[:split]
y_train = y[:split]
# Test data set
X_test = X[split:]
y_test = y[split:]

# Fit a model on train data
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5,
                             class_weight='balanced')
clf = clf.fit(X_train, y_train)

# Use the trained model to make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model performance
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        29
           1       0.28      0.88      0.42         8
           2       0.00      0.00      0.00         1
           3       0.07      1.00      0.13         1

    accuracy                           0.21        39
   macro avg       0.09      0.47      0.14        39
weighted avg       0.06      0.21      0.09        39



You can try this model yourself on a new dataset to see how it works. In the next unit, there will be an interactive exercise. All the best!